In [1]:
import pandas as pd

In [2]:
df_original = pd.read_csv('Patient Matching Data.csv')

In [3]:
df_original

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,...,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,1,1,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,1,2,NaN,Suttin,James,Power,9/21/1945,Male,1859 Sullivan Parkway,#2,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,1,3,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,...,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,NaN,Sutton,NaN,Power,9/20/1954,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,NaN,SUTTON,NaN,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,61,197,NaN,Bill,NaN,Smith,5/16/1972,Male,16595 City View Lane,NaN,...,WA,98684.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,62,198,NaN,Bill,NaN,Smith,5/16/1972,Male,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,63,199,NaN,Sarah,NaN,Field,3/17/2010,Female,9850 Kelso Road,NaN,...,WA,98626.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,64,200,NaN,Sara,NaN,Field,3/17/2010,Female,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_original = df_patients.copy()

In [7]:
# idx???
df_original

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,...,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,1,1,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,1,2,NaN,Suttin,James,Power,9/21/1945,Male,1859 Sullivan Parkway,#2,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,1,3,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,...,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,NaN,Sutton,NaN,Power,9/20/1954,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,NaN,SUTTON,NaN,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,61,197,NaN,Bill,NaN,Smith,5/16/1972,Male,16595 City View Lane,NaN,...,WA,98684.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,62,198,NaN,Bill,NaN,Smith,5/16/1972,Male,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,63,199,NaN,Sarah,NaN,Field,3/17/2010,Female,9850 Kelso Road,NaN,...,WA,98626.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,64,200,NaN,Sara,NaN,Field,3/17/2010,Female,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
for x in range(len(df_original)):
    dates = df_original['Date of Birth'][x].split('/')
    if len(dates[0]) == 1 and len(dates[1]) == 1:
        df_cleaned['Date of Birth'][x] = '0' + str(dates[0]) + '0' + str(dates[1]) + dates[2]
    elif len(dates[0]) == 1 and len(dates[1]) == 2:
        df_cleaned['Date of Birth'][x] = '0' + str(dates[0]) + str(dates[1]) + dates[2]
    elif len(dates[0]) == 2 and len(dates[1]) == 1:
        df_cleaned['Date of Birth'][x] = str(dates[0]) + '0' + str(dates[1]) + dates[2]
    else:
        df_cleaned['Date of Birth'][x] = str(dates[0]) + str(dates[1]) + dates[2]

df_cleaned

c:\users\jeremy\pycharmprojects\la_hacks_2019\venv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\jeremy\pycharmprojects\la_hacks_2019\venv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\jeremy\pycharmprojects\la_hacks_2019\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,...,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,1,1,247028705-7,Sutton,J,Power,09201945,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,1,2,NaN,Suttin,James,Power,09211945,Male,1859 Sullivan Parkway,#2,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,1,3,247028705-7,Sutton,J,Power,09201945,Male,1858 Sullivan Parkway,NaN,...,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,NaN,Sutton,NaN,Power,09201954,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,NaN,SUTTON,NaN,POWER,09201954,Male,1858 SULLIVAN PKWAY,APT 2,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,61,197,NaN,Bill,NaN,Smith,05161972,Male,16595 City View Lane,NaN,...,WA,98684.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,62,198,NaN,Bill,NaN,Smith,05161972,Male,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,63,199,NaN,Sarah,NaN,Field,03172010,Female,9850 Kelso Road,NaN,...,WA,98626.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,64,200,NaN,Sara,NaN,Field,03172010,Female,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# can these be mistyped like names? 
df_cleaned['Sex'].fillna('U', inplace = True)
df_cleaned['Sex'].replace('Male', 'M', inplace = True)
df_cleaned['Sex'].replace('Female', 'F', inplace = True)
df_cleaned

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,...,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,1,1,247028705-7,Sutton,J,Power,09201945,M,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,1,2,NaN,Suttin,James,Power,09211945,M,1859 Sullivan Parkway,#2,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,1,3,247028705-7,Sutton,J,Power,09201945,M,1858 Sullivan Parkway,NaN,...,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,NaN,Sutton,NaN,Power,09201954,M,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,NaN,SUTTON,NaN,POWER,09201954,M,1858 SULLIVAN PKWAY,APT 2,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,61,197,NaN,Bill,NaN,Smith,05161972,M,16595 City View Lane,NaN,...,WA,98684.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,62,198,NaN,Bill,NaN,Smith,05161972,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,63,199,NaN,Sarah,NaN,Field,03172010,F,9850 Kelso Road,NaN,...,WA,98626.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,64,200,NaN,Sara,NaN,Field,03172010,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
#maybe we shouldn't remove nonalphanumeric (account for it?)
df_cleaned['First Name'].str.replace('[^a-zA-Z]', '').str.lower()

0      sutton
1      suttin
2      sutton
3      sutton
4      sutton
        ...  
196      bill
197      bill
198     sarah
199      sara
200      sara
Name: First Name, Length: 201, dtype: object

In [35]:
df_cleaned['Last Name'].str.replace('[^a-zA-Z]', '').str.lower()
df_cleaned

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,...,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,1,1,247028705-7,Sutton,J,Power,09201945,M,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,1,2,NaN,Suttin,James,Power,09211945,M,1859 Sullivan Parkway,#2,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,1,3,247028705-7,Sutton,J,Power,09201945,M,1858 Sullivan Parkway,NaN,...,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,NaN,Sutton,NaN,Power,09201954,M,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,NaN,SUTTON,NaN,POWER,09201954,M,1858 SULLIVAN PKWAY,APT 2,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,61,197,NaN,Bill,NaN,Smith,05161972,M,16595 City View Lane,NaN,...,WA,98684.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,62,198,NaN,Bill,NaN,Smith,05161972,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,63,199,NaN,Sarah,NaN,Field,03172010,F,9850 Kelso Road,NaN,...,WA,98626.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,64,200,NaN,Sara,NaN,Field,03172010,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
